In [ ]:
import random
import numpy as np
import copy as cp
from scipy.special import comb


class Board:
    def __init__(self, n):
        self.n_queen = n
        self.map = [[0 for j in range(n)] for i in range(n)]


    def get_dim(self) :
      return self.n_queen


    def get_queen(self, r) :
      '''
      Retrieves column position of queen in row 'r' 
      '''
      c = 0
      #
      for e in self.map[r] :
        if e == 1 :
          return c
        else :
          c += 1


    def queen_on(self, r, c) :
      '''
      Place a queen at row 'r' column 'c'
      '''
      self.map[r][c] = 1

  
    def queen_off(self, r, c) :
      '''
      Remove queen from row 'r' column 'c'
      '''
      self.map[r][c] = 0


    def set_queens(self) :
        for i in range(self.n_queen):
            j = random.randint(0, self.n_queen - 1)
            self.map[i][j] = 1
 

    def reset_row(self, r) :
      '''
      Move queen to column 0 in row 'r'
      '''
      self.map[r][self.get_queen(r)] = 0
      self.map[r][0] = 1


    def fitness(self):
        fit = self.n_queen * (self.n_queen - 1) // 2
        for i in range(self.n_queen):
            for j in range(self.n_queen):
                if self.map[i][j] == 1:
                    for k in range(1, self.n_queen - i):
                        if self.map[i + k][j] == 1:
                            fit -= 1
                        if j - k >= 0 and self.map[i + k][j - k] == 1:
                            fit -= 1
                        if j + k < self.n_queen and self.map[i + k][j + k] == 1:
                            fit -= 1
        return fit


    def show(self):
        print(np.matrix(self.map))
        print( "Fitness: ",  self.fitness(),"\n" )
#
#
#       
if __name__ == '__main__':
    test = Board(5)
    test.set_queens()
    test.show()    

In [ ]:
#
#
#
class Hill_Climb:
    '''
    @param b : pre-initialized Board object to be optimized
    @param n : max number of allowed steps
    '''
    def __init__(self, b, n) :
        self.step = 0
        self.board = b
        self.max_steps = n


    def optimize(self) :
        '''
        Attempts to find a solution within "self.max_steps" steps
        '''
        best_fit = self.board.fitness()
        best_clmns = []
        # initialize optimal solution to original config
        for r in range( self.board.get_dim() ) :
            best_clmns.append( self.board.get_queen(r) )
        # loop until soln found or max_steps taken
        while ( best_fit < comb(self.board.get_dim(), 2, exact=True) ) and (self.step < self.max_steps) :
            # step through each row
            for r in range( self.board.get_dim() ) :
                # memorize original column position
                pos0 = cp.deepcopy( best_clmns[r] )
                # pick queen up
                self.board.queen_off(r, self.board.get_queen(r) )
                # slide queen along row
                for c in range( self.board.get_dim() ) :
                    # put queen down
                    self.board.queen_on(r, c)
                    if self.board.fitness() == comb(self.board.get_dim(), 2, exact=True) :
                        best_clmns[r] = c
                        self.step += 1
                        return 
                    elif self.board.fitness() > best_fit :
                        # save column with better fit
                        best_clmns[r] = c
                        best_fit = self.board.fitness()
                    # pick queen up  
                    self.board.queen_off(r, c)
                # return queen to best column before moving to next row
                self.board.queen_on( r, best_clmns[r] )
                self.step += 1  
                print("STEP:", self.step)
                self.board.show()


    def show(self) :
        self.board.show()     
        print("\nSteps:", self.step)  
#
#
#
if __name__ == '__main__':
    board = Board(5)
    board.set_queens()
    board.fitness()
    print("INITIAL CONFIGURATION")
    board.show() 

    climb = Hill_Climb(board, 10000)
    climb.optimize()
    print("FINAL CONFIGURATION")
    climb.show()

In [ ]:
class Genetic:
  def __init__(self, dim, k, n):
    self.step = 0
    self.max_steps = n
    self.k_states = []
    # initialize k_states
    for i in range(k) :
      board = Board(dim)
      board.set_queens()
      self.k_states.append( self.encode(board) )


  def find_soln(self) :
    '''
    Performs selection, pairing, crossover and mutation. Returns configuration
    optimal fitness score or the highest fitness score after 'max_steps' 
    iterations. 
    '''
    #
    optimal = comb(len( self.k_states[0] ), 2, exact=True) 
    #
    best = self.best_fit(self.k_states)
    #
    while (best.fitness() < optimal) and (self.step < self.max_steps) :
      cum_prob = self.calc_fit(self.k_states)
      #
      select_states = self.selection(self.k_states, cum_prob)
      #
      cross_states = self.pair_and_cross(select_states)
      #
      mutants = self.mutate(cross_states)
      #
      candidate = self.best_fit(mutants)
      if candidate.fitness() > best.fitness() :
        best = candidate
      #
      self.k_states = mutants
      #
      self.step += 1
    #
    return best


  def best_fit(self, states) :
    '''
    Calculates fitness scores for each configuration in 'states' list and 
    returns the configuration with the highest score.
    '''
    #
    best = 0
    soln = 0
    #
    for i in range( len(states) ) :
      board = self.decode( states[i] )
      fit = board.fitness()
      #
      if fit > best :
        soln = board
        best = fit
 
    return soln


  def mutate(self, states) :
    '''
    For each configuration in 'states', randomly chooses an element 
    (row position) and changes the value (column position).
    '''
    mutants = []

    for i in range( len(states) ) :
      rand_pos = np.random.randint( 0, len(states[0]) )
      rand_val = np.random.randint( 0, len(states[0]) )

      states[i][rand_pos] = rand_val
      mutants.append( cp.deepcopy( states[i] ) )
    
    return mutants


  def pair_and_cross(self, states) :
    '''
    Pairs configurations in 'states' list, randomly slices each pair at a 
    common index, then swaps the tail ends of each configuration string in the 
    pair. 
    '''
    cross_states = []

    for i in range(len(states)//2) :
      demarc = np.random.randint( 0, len(states[0]) ) 
      
      s1 = states[2*i][0:demarc+1] + states[2*i+1][demarc+1:] 
      s2 = states[2*i+1][0:demarc+1] + states[2*i][demarc+1:]

      cross_states.append(s1)
      cross_states.append(s2)
    
    return cross_states


  def selection(self, states, cum_prob) :
    '''
    For each configuration in 'states', generates a random number in range (0,1)
    and selects the corresponding configuration mapped to 'cum_prob'. 
    '''
    new_states = []    

    for i in range( len(states) ) : 
      rand = np.random.rand()

      for j in range( len(cum_prob) ) :
        if rand < cum_prob[j] :
          new_states.append( states[j] )
          break

    return new_states


  def calc_fit(self, states) :  
    '''
    Transforms each configuration in 'states' to a Board object, calculates its
    fitness score and constructs then returns a cumulative probability array. 
    '''
    fit_list = []
    prob_list = []
    cum_prob = []

    for r in range( len(states) ) :
      fit_list.append( self.decode( states[r]).fitness() )

    for r in range( len(fit_list) ) :
      prob_list.append( fit_list[r] / sum(fit_list) )

    accum = 0
    for r in range( len(prob_list) ) :
      cum_prob.append( accum + prob_list[r] ) 
      accum += prob_list[r]

    return cum_prob


  def encode(self, b) :
    '''
    Represents a chessboard of Queens as a list of integers. The ith integer 'c' 
    in the list means row 'i' of the chessboard has a Queen in column 'c'.

    @param b : Board object to be encoded. 

    @returns : integer list
    '''
    state = []

    for r in range( b.get_dim() ) :
      state.append( b.get_queen(r) )

    return cp.deepcopy(state)


  def decode(self, state) :
    '''
    Transforms state configuration into a board object and returns it
    '''
    new_board = Board( len(state) )

    for r in range( len(state) ) : 
      new_board.queen_on( r, state[r] )

    return new_board

  
  def show_k_states(self) :
    print( np.matrix(self.k_states) )
#
#
#
if __name__ == '__main__':
  gene = Genetic(5, 8, 10000)
  gene.find_soln().show()
  print("STEPS:", gene.step)